In [1]:
import numpy as np

import torch
from torch.utils.data import DataLoader

from deeptime.models.representation import ConvAutoEncoder
from deeptime.data import BaseDataset

from sktime.datasets import load_UCR_UEA_dataset

from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
DATASETS = [
    'Yoga',
    # 'WormsTwoClass',
    # 'Wine',
    # 'Wafer',
    # 'TwoLeadECG',
    # 'Strawberry',
    # 'SemgHandGenderCh2', 
    # 'BeetleFly',
    # 'BirdChicken',
    # 'Computers',
    # 'DistalPhalanxOutlineCorrect',
    # 'Earthquakes',
    # 'ECG200',
    # 'ECGFiveDays',
    # 'FordA',
    # 'FordB',
    # 'HandOutlines',
    # 'ItalyPowerDemand', 
    # 'MiddlePhalanxOutlineCorrect',
    # 'Chinatown',
    # 'FreezerRegularTrain',
    # 'FreezerSmallTrain',
    # 'GunPointAgeSpan',
    # 'GunPointMaleVersusFemale',
    # 'GunPointOldVersusYoung',
    # 'PowerCons',
    # 'Coffee',
    # 'Ham',
    # 'Herring',
    # 'Lightning2',
    # 'MoteStrain',
    # 'PhalangesOutlinesCorrect',
    # 'ProximalPhalanxOutlineCorrect',
    # 'ShapeletSim',
    # 'SonyAIBORobotSurface1',
    # 'SonyAIBORobotSurface2',
    # 'ToeSegmentation1',
    # 'ToeSegmentation2',
    # 'HouseTwenty'
]

In [4]:
for dataset in DATASETS:
    print(f'\tDownloading the {dataset} dataset...')
    x_train, y_train = load_UCR_UEA_dataset(name=dataset, split='train')
    # Since the features from the sktime are instatiated as objects we have to manually convert them
    y_train = np.array(y_train, dtype=np.int32)

    sequence_length = x_train.values[0][0].shape[0]

    x_test, y_test = load_UCR_UEA_dataset(name=dataset, split='test')
    y_test = np.array(y_test, dtype='int32') # Fixing the labels type
    print(f'\t\tDownload completed!')

    print('\tTransforming data to the correct format...')
    x_train_transformed = []
    for val in x_train.values:
        x_train_transformed.append(val[0].tolist())
    x_train = np.array(x_train_transformed)

    x_test_transformed = []
    for val in x_test.values:
        x_test_transformed.append(val[0].tolist())
    x_test = np.array(x_test_transformed)
    
    x_train = np.expand_dims(x_train_transformed, axis=1)
    x_test = np.expand_dims(x_test_transformed, axis=1)
    print('\t\tTransform completed!')

    print('\tStarting the classifier...')
    unique_labels = np.unique(y_train)

    model = ConvAutoEncoder(
        in_channels=1,
        in_features=sequence_length,
        latent_dim=32,
    )

    model.load_state_dict(
        torch.load(f'../../../pretrain/representation/{dataset}/conv_autoencoder.pt')
    )

    for label in unique_labels:
        print(f'\t\tClassifying the label {label}...')
        
        x_train_ = x_train[y_train == label]
        y_train_ = y_train[y_train == label]

        occ_labels = [1 if x == label else -1 for x in y_test]

        train_dataset = BaseDataset(x=x_train, y=y_train)
        train_loader = DataLoader(train_dataset, batch_size=32)
        
        representations = []
        for x, y in train_loader:
            x_hat, z = model(x)
            representations.extend(z.tolist())

        representations = np.array(representations)
        # print(representations.shape)
        # representations = representations.reshape((representations.shape[0], representations.shape[2]))

        clf = OneClassSVM(gamma='scale', nu=0.1, kernel='rbf').fit(representations)
        
        test_dataset = BaseDataset(x=x_test, y=y_test)
        test_loader = DataLoader(test_dataset, batch_size=32)

        test_representations = []

        for x, y in test_loader:
            x_hat, z = model(x)
            test_representations.extend(z.tolist())

        test_representations = np.array(test_representations)
        # test_representations = test_representations.reshape((test_representations.shape[0], test_representations.shape[2]))

        result_labels = clf.predict(test_representations)
        
        print(f'\t\tReport:')
        # print(f'\t\t\tAccuracy Score = {accuracy_score(occ_labels, result_labels)}')
        print(f'\t\t\tF1 Score = {f1_score(occ_labels, result_labels)}')

    print('-' * 100)

		Download completed!
	Transforming data to the correct format...
		Transform completed!
	Starting the classifier...
		Classifying the label 1...
		Report:
			F1 Score = 0.6114920785055569
		Classifying the label 2...
		Report:
			F1 Score = 0.6945757371145622
----------------------------------------------------------------------------------------------------
